In [ ]:
# Create train and test set and save csv's.
# Takes 1,5 min

# import pandas as pd
# from sklearn.model_selection import train_test_split

# df_music = pd.read_csv('data/music_data.csv')
# df_meta = pd.read_csv('data/metadata.csv')

# train, test = train_test_split(df_music, test_size=0.2, random_state=1)

# train.to_csv('data/music_data_train.csv')
# test.to_csv('data/music_data_test.csv')
# df_meta.iloc[train.index].to_csv('data/metadata_train.csv', index=False)
# df_meta.iloc[test.index].to_csv('data/metadata_test.csv', index=False)

# train test sampling with respect to balancing cats??
